In [1]:
#インポート
if 1:
  import numpy as np
  import matplotlib.pyplot as plt
  import cv2
  import os
  import pickle
  from google.colab import drive
  from google.colab.patches import cv2_imshow
  from scipy import io
  from math import *
  from sklearn.model_selection import train_test_split

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#正規化
def z_score(t,data):
  out=np.zeros([len(data),len(data[0]),len(data[0][0])])
  r0=0
  r1=0

  data=data.transpose(1,0,2)
  data-=np.sum(data,axis=0)/len(data)#平均０
  data/=np.sqrt(np.sum(data*data,axis=0)/len(data))#分散１
  data=data.transpose(1,0,2)
  out=data
  #plt.show()
  return t,out

#最小二乗法によるベースライン補正
def base_line(t,data):
  data=data.transpose(0,2,1)

  a=np.zeros([2,len(data),len(data[0])])
  out=np.zeros([len(data),len(data[0]),len(data[0][0])])

  A=np.sum(data*t,axis=2)/len(t)#sum_xy
  B=np.sum(data,axis=2)/len(t)#sum_y
  C=np.sum(t*t,axis=0)/len(t)#sum_xx
  D=np.sum(t,axis=0)/len(t)#sum_x

  a[0]=(B*C-A*D)/(C-D*D)
  a[1]=(A-B*D)/(C-D*D)

  data=data.transpose(2,0,1)
  out=out.transpose(2,0,1)

  for time in range(len(t)):
    out[time]=data[time]-(a[0]+a[1]*t[time])
  
  data=data.transpose(1,0,2)
  out=out.transpose(1,0,2)
  
  return t,out

#フーリエ変換
def fourier(t,data):
  n_sam=1500
  out=np.zeros([len(data),len(data[0]),len(data[0][0])])
  
  F=(np.fft.fft(data,n_sam,axis=1,norm="ortho")).real
  freq=np.fft.fftfreq(n_sam,1/len(t))

  F=F.transpose(1,0,2)
  F=F[0:floor(8/len(t)*n_sam)]
  F=F.transpose(1,0,2)

  freq=freq[0:floor(8/len(t)*n_sam)]

  return freq,F

#試行加算平均（セッション別）
def ave_per_ses(data,label,num_ses,n_class):
  m_ses=12
  out_data=np.zeros([n_class*m_ses,len(data[0]),len(data[0][0])])
  n_label=np.zeros([n_class*m_ses])

  for trial in range(len(data)):#データ
    if n_class==2:
      ind=(label[trial][0]+1)/2
    else:
      ind=label[trial][0]-1
      
    ind=int(ind+n_class*(num_ses[trial][0]-1))
    n_label[ind]+=1
    out_data[ind]+=data[trial]
  
  out_data=out_data.transpose(2,1,0)
  out_data/=n_label
  out_data=out_data.transpose(2,1,0)

  out_label=np.zeros([n_class*m_ses,1])#ラベル
  for trial in range(n_class*m_ses):
    if n_class==2:
      out_label[trial]=(trial%n_class)*2-1
    else:
      out_label[trial]=(trial%n_class)+1
  return out_data,out_label

# ラベル変形
def remake_label(label,n_class):
  label2=np.zeros([len(label),n_class])

  
  for n_label in range(len(label)):
    if n_class==2:
      cl=int((label[n_label]+1)/2)
    else:
      cl=int(abs(label[n_label]-1))
    label2[n_label][cl]=1

  return label2

#データセット編集(ロード時)
def arange_data(data_prepro,label,num_ses,n_class):
  t=np.arange(0,1,1/256)
  t,data=base_line(t,data_prepro)
  t,data=z_score(t,data)
  #freq,data=fourier(t,data)
  data,label=ave_per_ses(data,label,num_ses,n_class)
  ##label=remake_label(label,n_class)
  
  #data,label=sub_trial_ave(data,label)
  #data=del_ch_ave(data)
  #print(data.shape,label.shape)
  #t,data=z_score(t,data)
  return data,label

def make_dataset(n_class):
  #データロード
  after_data = []
  after_label = []
  for people in range(1,10):#max=10
    matdata_org = io.loadmat("/content/drive/Shareddrives/和田・南部研究室共有ドライブ/datasets/eeg_sound/s"+str(people)+"_500.mat")# loading data (for 500ms SOA)
    
    data_prepro =matdata_org['DataF']# pre-proceed data (subtrials x time x channels)
    label_code =matdata_org['Code']# sound direction (1-6, labels for 6-class classification)
    label_target =matdata_org['Group'] #target-nontarget labels (1:target, -1:nontarget)
    #num_rep = matdata_org['Rep']# repetition number (= trial number)
    num_ses = matdata_org['Session']# Session number (= session number)

    if n_class==2:
      label_0=label_target
    else:
      label_0=label_code

    if people ==1:
      data_1,label_1=arange_data(data_prepro,label_0,num_ses,n_class)
    else:
      data_0,label_0=arange_data(data_prepro,label_0,num_ses,n_class)
      data_1=np.concatenate([data_1,data_0])
      label_1=np.concatenate([label_1,label_0])
    #print(data_1.shape,label_1.shape)
    after_data.append(data_1)
    after_label.append(label_1)
  #データセット編集（ロード後)
  data,label=data_1,label_1
  t=np.arange(0,1,1/256)
  #t,data=base_line(t,data_prepro)
  #t,data=z_score(t,data)
  freq,data=fourier(t,data)
  #data,label=sub_trial_ave(data,label)
  #data=del_ch_ave(data)
  #print(data.shape,label.shape)
  t,data=z_score(t,data)

  return after_data,after_label

In [4]:
data,label=make_dataset(2)#output preprocessed data and label
data1,data2,data3,data4,data5,data6,data7,data8,data9 = data
label1,label2,label3,label4,label5,label6,label7,label8,label9 = label
D  = [data1,data2,data3,data4,data5,data6,data7,data8,data9]
L = [label1,label2,label3,label4,label5,label6,label7,label8,label9]


In [5]:
print(D[0].shape)
datax = D[0].reshape(D[0].shape[0],D[0].shape[1]*D[0].shape[2])
print(datax.shape)

(24, 256, 64)
(24, 16384)


In [6]:
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
def split(data,label):  #Use Decision Tree to calculate test size optimally.
  a = np.array([])
  K = [0.1, 0.2, 0.3, 0.4, 0.5]
  datax = data.reshape(data.shape[0],data.shape[1]*data.shape[2])
  labelx = label.flatten()
  for test_split in K:
    #print("Percentage of test data in datasets % :", test_split)
    X_train, X_test, y_train, y_test = train_test_split(datax, labelx, test_size= test_split, random_state=0)
    #print("Train data shape                      :", X_train.shape, y_train.shape)
    #print("           label counts               :", Counter(y_train))
    #print("Test  data shape                      :", X_test.shape , y_test.shape)
    #print("           label counts               :", Counter(y_test))
    
    clf = tree.DecisionTreeClassifier(random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)
    #print("Model accuracy (training)             :", accuracy_score(y_train, y_pred))

    # make predictions using test data
    y_test_pred = clf.predict(X_test)
    a = np.append(a, accuracy_score(y_test, y_test_pred))
    # calculate test data accuracy
    print("Model accuracy (testing)              :", accuracy_score(y_test, y_test_pred))
    
  print('Max accuracy Test :',max(a))
  i = int(np.where(a == max(a))[0][0])
  print('Best Percentage of test data % :',K[i])
  print('-------------------------------------')
  return K[i]
spl1= []
for i in range(0,9):
  print('S'+str(i+1)+'_500.mat')
  spl = split(D[i],L[i])
  spl1.append(spl)

S1_500.mat
Model accuracy (testing)              : 1.0
Model accuracy (testing)              : 1.0
Model accuracy (testing)              : 0.625
Model accuracy (testing)              : 0.7
Model accuracy (testing)              : 0.75
Max accuracy Test : 1.0
Best Percentage of test data % : 0.1
-------------------------------------
S2_500.mat
Model accuracy (testing)              : 1.0
Model accuracy (testing)              : 1.0
Model accuracy (testing)              : 0.8666666666666667
Model accuracy (testing)              : 0.9
Model accuracy (testing)              : 0.9166666666666666
Max accuracy Test : 1.0
Best Percentage of test data % : 0.1
-------------------------------------
S3_500.mat
Model accuracy (testing)              : 1.0
Model accuracy (testing)              : 1.0
Model accuracy (testing)              : 1.0
Model accuracy (testing)              : 1.0
Model accuracy (testing)              : 0.75
Max accuracy Test : 1.0
Best Percentage of test data % : 0.1
--------------

In [7]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in range(0,9):
  #Create Train dataset and Test dataset with Test_size = spl
  datax = D[i].reshape(D[i].shape[0],D[i].shape[1]*D[i].shape[2])
  labelx = L[i].flatten()
  X_tr, X_t, y_tr, y_t = train_test_split(datax, labelx, test_size= spl1[i], random_state=0)
  X_train.append(X_tr)
  X_test.append(X_t)
  y_train.append(y_tr)
  y_test.append(y_t)
print(y_train[1].shape)
print(X_train[1].shape)

(43,)
(43, 16384)


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pandas as pd
def parameters(X_train1,y_train1):  #Finding the best Hyperparameter tuning in SVM (in case the kernel is linear)
  param_grid = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']}]
  #print(param_grid)
  grid = GridSearchCV(SVC(),param_grid, cv=5, scoring='accuracy')  #10-Fold
  grid.fit(X_train1, y_train1)
  df = pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
  print(df)
  print('\nMax_score = ',grid.best_score_)
  print('Estimator : ',grid.best_estimator_)
  print('-------------------------------------')
  return (100*grid.best_score_)
cross_acc=[]
for i in range(0,9):
  print('S'+str(i+1)+'_500.mat')
  para = parameters(X_train[i],y_train[i])
  cross_acc.append(para)
print('Cross validation accuracy:  %f %%' %np.mean(cross_acc))

S1_500.mat
   mean_test_score  std_test_score                           params
0             0.96            0.08     {'C': 1, 'kernel': 'linear'}
1             0.96            0.08    {'C': 10, 'kernel': 'linear'}
2             0.96            0.08   {'C': 100, 'kernel': 'linear'}
3             0.96            0.08  {'C': 1000, 'kernel': 'linear'}

Max_score =  0.96
Estimator :  SVC(C=1, kernel='linear')
-------------------------------------
S2_500.mat
   mean_test_score  std_test_score                           params
0              1.0             0.0     {'C': 1, 'kernel': 'linear'}
1              1.0             0.0    {'C': 10, 'kernel': 'linear'}
2              1.0             0.0   {'C': 100, 'kernel': 'linear'}
3              1.0             0.0  {'C': 1000, 'kernel': 'linear'}

Max_score =  1.0
Estimator :  SVC(C=1, kernel='linear')
-------------------------------------
S3_500.mat
   mean_test_score  std_test_score                           params
0              1.0          

In [9]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import classification_report
def SVM(X_train1,y_train1,X_test1,y_test1,spl2,datay,labely):
  print('In Datasets, %d %% is Train data and %d %% is Test data.' %(((1-spl2)*100),(spl2*100)))
  print('--------------------------------------------------------')
  clf = SVC(kernel='linear',C =1)
  clf.fit(X_train1,y_train1) 
  w_sklearn = clf.coef_.reshape(-1, 1)
  b_sklearn = clf.intercept_[0]
  y_pred = clf.predict(X_test1)
  print('\nw = ',w_sklearn.T)
  print('b = ',b_sklearn)
  accuracy = 100*accuracy_score(y_test1, y_pred)
  print("\nAccuracy: %.2f %%" %(accuracy))
  print(classification_report(y_test1, y_pred))

  print(X_test1.shape, y_pred.shape)
  kf= KFold(n_splits= 5)  #cross-validation with 5-Fold
  score= 100*cross_val_score(clf,X_train1,y_train1,cv=kf)
  #print("Cross Validation Scores: {}".format(score))
  print("Average Cross Validation score: {}".format(score.mean()))
  print('=========================================================\n')
  return accuracy,score.mean()

acc = [] #Store accuracy data to ndarray
cracc = []  #Store Cross Validation accuracy data to ndarray
for i in range(0,9):
  print('S'+str(i+1)+'_500.mat')
  svm = SVM(X_train[i],y_train[i],X_test[i],y_test[i],spl1[i],D[i],L[i])
  acc.append(svm[0])
  cracc.append(svm[1])
print('Average of accuracy:  %.2f %%' %np.mean(acc))
print('Cross-Validation accuracy: %.2f %%' %np.mean(cracc))

S1_500.mat
In Datasets, 90 % is Train data and 10 % is Test data.
--------------------------------------------------------

w =  [[ 5.32810233e-05  1.72448745e-04 -2.86716078e-04 ... -6.66440545e-04
  -4.88178560e-04 -3.66886916e-04]]
b =  -0.6681761931690576

Accuracy: 100.00 %
              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00         2
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

(3, 16384) (3,)
Average Cross Validation score: 96.0

S2_500.mat
In Datasets, 90 % is Train data and 10 % is Test data.
--------------------------------------------------------

w =  [[-0.00014712 -0.00018127 -0.00054941 ... -0.00060888 -0.00051825
  -0.00032771]]
b =  -0.6300388397580895

Accuracy: 100.00 %
              precision    recall  f1-score   support

        -1.0       1.00     